# IBM Data Science Capstone Project Problem Definition and Data

## Problem definition

#### A theoretical business problem was chosen for this project. The question to be answered is the following.

     A successful owner of multiple mid to high-end restaurants decided to open a new restaurant in São Paulo, Brazil. Having visited the city many times in recent yearsm he couldn't disregard the big boom in gastronomy. He is keen on opening a new unit, which will focus on the Italian kitchen.
     Taking in account the price level at which the restaurants will operate, the intent is to find an optimal location in an aream where gastronomy is booming and which is easily accessible for tourists and for weaslthier local citizens as well.

## Assumptions, business logic

    The assumption behind the analysis is that we can use unsupervised machine learning to create cluesters of districts that will provide us with a lista of areas for consideration for the restaurant. The intent is that the restaurant to be situated close to one of the gastronomical centres.
    
## Data

    To perform this analysis, we will need the following data:
    
    1. Lis of the districts of São Paulo
    2. Geo-coordinates of the districts in São Paulo
    3. Top venues of districts.
    
    Lis of district will be obtained from wikipedia:                     

        https://pt.wikipedia.org/wiki/Lista_dos_distritos_de_São_Paulo_por_população


    Geo-coordinates of districts will be obtained with the help of the geocoder tool in the notebook.
    Top venues data will be obtained from Foursquare through an API.

## Use of Data and Methodology

    After tidyinf up and exploring the data, we will apply the K-means machine learning technique for creating cluster of districts. We will use the silhouette score for choosing the optimal number of clusters.